In this dataset a classifier will be trained on N=800, evaluated in-domain, and text, labels and predictions will be saved for domain expert examination.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import os
import gc
import parse
import logging
import numpy as np
import pandas as pd

from utils import get_N_tokens, read_and_split_file, raw_dir, interim_dir, final_dir, load_SET_dataset, LABELS

logging.basicConfig(format="%(asctime)s - %(message)s", level=logging.INFO)


texts, labels = list(), list()

files = ["bswac_tail_pp", "cnrwac_tail_pp_corrected_2",
         "hrwac_tail_pp", "srwac_tail_pp"]

langs = ["bs", "me", "hr", "sr"]

for file, lang in zip(files, langs):
    full_path = os.path.join(interim_dir, file)
    current_texts = read_and_split_file(full_path)
    len_cur_texts = len(current_texts)
    texts.extend(current_texts)
    labels.extend([lang] * len_cur_texts)

dataset = pd.DataFrame(data={"text": texts, "labels": labels})
train, eval_df = train_test_split(dataset, test_size=0.1, stratify=dataset.labels)


clf = GaussianNB()

# Create tokenizer:
N = 800
vocabulary = get_N_tokens(N)
vectorizer = CountVectorizer(
    vocabulary=vocabulary, lowercase=True, binary=True)

train_vectors = vectorizer.fit_transform(train.text)
train_labels = train.labels
test_vectors = vectorizer.fit_transform(eval_df.text)
y_true = eval_df.labels


clf.fit(train_vectors.toarray(), train_labels)
y_pred = clf.predict(test_vectors.toarray())

eval_df["predicted"] = y_pred


2022-01-14 16:01:49,806 - NumExpr defaulting to 8 threads.
<ipython-input-1-32396805b000>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df["predicted"] = y_pred


In [2]:
def clean_whitespace(s: str) -> str:
    return " ".join(s.split())

eval_df["text"] = eval_df.text.apply(clean_whitespace)
eval_df.rename(columns={"labels": "gold"}, inplace=True)

<ipython-input-2-ced641e97915>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df["text"] = eval_df.text.apply(clean_whitespace)
/home/peterr/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
eval_df.loc[:, ["gold", "predicted", "text"]].to_csv("21_text_labels_predictions.csv", sep="\t", index=False)